In [157]:
# Установка необходимых библиотек
!pip install num2words transformers torch datasets scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import operator
from num2words import num2words #i'm not sure that it works with ru-version
import gc
import pandas as pd
from transformers import T5ForConditionalGeneration, GPT2Tokenizer
import torch
import logging
import re

In [6]:
INPUT_PATH = r'../data'
DATA_INPUT_PATH = r'../data/ru_with_types'
SUBM_PATH = INPUT_PATH

SUB = str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789")
SUP = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")
OTH = str.maketrans("፬", "4")

In [11]:

file = "ru_train.csv"
train = open(os.path.join(INPUT_PATH, "ru_train.csv"), encoding='UTF8')
line = train.readline()
res = dict()
total = 0
not_same = 0
while 1:
    line = train.readline().strip()
    if line == '':
        break
    total += 1
    pos = line.find('","')
    text = line[pos + 2:]
    if text[:3] == '","':
        continue
    text = text[1:-1]
    arr = text.split('","')
    if arr[0] != arr[1]:
        not_same += 1
    if arr[0] not in res:
        res[arr[0]] = dict()
        res[arr[0]][arr[1]] = 1
    else:
        if arr[1] in res[arr[0]]:
            res[arr[0]][arr[1]] += 1
        else:
            res[arr[0]][arr[1]] = 1
train.close()
print(file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))




ru_train.csv:	Total: 10574516 Have diff value: 1322698


In [12]:
files = os.listdir(DATA_INPUT_PATH)
corrupted_files = []
for file in files:
    if file.startswith(".ipynb_checkpoints"):
      continue
    print('reading file ' + file)
    train = open(os.path.join(DATA_INPUT_PATH, file), encoding='UTF8')
    while 1:
      try:
          line = train.readline().strip()
          if line == '':
              break
          total += 1
          pos = line.find('\t')
          text = line[pos + 1:]
          if text[:3] == '':
              continue
          arr = text.split('\t')
          if arr[0] == '<eos>':
              continue
          if arr[1] != '<self>':
              not_same += 1

          if arr[1] == '<self>' or arr[1] == 'sil':
              arr[1] = arr[0]

          if arr[1] == '<self>' or arr[1] == 'sil':
              arr[1] = arr[0]

          if arr[0] not in res:
              res[arr[0]] = dict()
              res[arr[0]][arr[1]] = 1
          else:
              if arr[1] in res[arr[0]]:
                  res[arr[0]][arr[1]] += 1
              else:
                  res[arr[0]][arr[1]] = 1
      except:
        print(file + " corrupted")
        corrupted_files.append(file)
        continue
    train.close()
    print(file + ':\tTotal: {} Have diff value: {}'.format(total, not_same))
    gc.collect()

reading file output-00000-of-00100
output-00000-of-00100:	Total: 13548162 Have diff value: 2325986
reading file output-00001-of-00100
output-00001-of-00100:	Total: 16529231 Have diff value: 3331025
reading file output-00002-of-00100
output-00002-of-00100:	Total: 19520932 Have diff value: 4334763
reading file output-00003-of-00100
output-00003-of-00100:	Total: 22477744 Have diff value: 5328391
reading file output-00004-of-00100
output-00004-of-00100:	Total: 25446261 Have diff value: 6328615
reading file output-00005-of-00100
output-00005-of-00100:	Total: 28421732 Have diff value: 7328628
reading file output-00006-of-00100
output-00006-of-00100:	Total: 31401788 Have diff value: 8333704
reading file output-00007-of-00100
output-00007-of-00100:	Total: 34377043 Have diff value: 9332343
reading file output-00008-of-00100
output-00008-of-00100:	Total: 37351670 Have diff value: 10334059
reading file output-00009-of-00100
output-00009-of-00100:	Total: 40318973 Have diff value: 11332092
reading 

In [13]:

sdict = {}
sdict['km2'] = 'квадратных километров'
sdict['km²'] = 'квадратных километров'
sdict['km'] = 'кидлметров'
sdict['kg'] = 'килограмм'
sdict['lb'] = 'фунтов'
sdict['dr'] = 'доктор'
sdict['m²'] = 'квадратных метров'
sdict['m2'] = 'квадратных метров'
sdict['км2'] = 'квадратных километров'
sdict['км²'] = 'квадратных километров'
sdict['км'] = 'кидлметров'
sdict['кг'] = 'килограмм'
sdict['м²'] = 'квадратных метров'
sdict['м2'] = 'квадратных метров'
sdict['#'] = 'номер'
sdict['№'] = 'номер'
sdict['%'] = 'процент'
sdict['久'] = 'х_trans и_trans с_trans а_trans'
sdict['石'] = 'и_trans с_trans и_trans'
sdict['譲'] = 'д_trans з_trans ё_trans'
sdict['千'] = 'т_trans и_trans'
sdict['と'] = 'т_trans о_trans'
sdict['尋'] = 'х_trans и_trans р_trans о_trans'
sdict['の'] = 'н_trans о_trans'
sdict['神'] = 'к_trans а_trans м_trans и_trans'
sdict['隠'] = 'к_trans а_trans к_trans у_trans с_trans и_trans'
sdict['し'] = 'с_trans и_trans'
sdict['イ'] = 'и_trans'
sdict['メ'] = 'м_trans э_trans'
sdict['ー'] = '-'
sdict['ジ'] = 'д_trans з_trans и_trans'
sdict['ア'] = 'а_trans'
sdict['ル'] = 'р_trans у_trans'
sdict['バ'] = 'б_trans а_trans'
sdict['ム'] = 'м_trans у_trans'

In [232]:
total = 0
changes = 0
out = open(os.path.join(SUBM_PATH, 'baseline_ext_ru.csv'), "w", encoding='UTF8')
out.write('"id","after"\n')
test = open(os.path.join(INPUT_PATH, "ru_test_2.csv"), encoding='UTF8')
line = test.readline().strip()
while 1:
    line = test.readline().strip()
    if line == '':
        break

    pos = line.find(',')
    i1 = line[:pos]
    line = line[pos + 1:]

    pos = line.find(',')
    i2 = line[:pos]
    line = line[pos + 1:]

    line = line[1:-1]
    out.write('"' + i1 + '_' + i2 + '",')
    if line in res:
        srtd = sorted(res[line].items(), key=operator.itemgetter(1), reverse=True)
        out.write('"' + srtd[0][0] + '"')
        changes += 1
    else:
        if len(line) > 1:
            val = line.split(',')
            if len(val) == 2 and val[0].isdigit and val[1].isdigit:
                line = ''.join(val)

        if line.isdigit():
            srtd = line.translate(SUB)
            srtd = srtd.translate(SUP)
            srtd = srtd.translate(OTH)
            out.write('"' + num2words(float(srtd)) + '"')
            changes += 1
        elif len(line.split(' ')) > 1:
            val = line.split(' ')
            for i, v in enumerate(val):
                if v.isdigit():
                    srtd = v.translate(SUB)
                    srtd = srtd.translate(SUP)
                    srtd = srtd.translate(OTH)
                    val[i] = num2words(float(srtd))
                elif v in sdict:
                    val[i] = sdict[v]

            out.write('"' + ' '.join(val) + '"')
            changes += 1
        else:
            out.write('"' + line + '"')

    out.write('\n')
    total += 1

print('Total: {} Changed: {}'.format(total, changes))
test.close()
out.close()

NameError: name 'res' is not defined

In [15]:
def dateFixer(x):
    return x.replace('two thousand and sixteen', 'две тысячи шестнадцатого').replace('two thousand and seventeen', 'две тысячи семнадцатого').replace('eleven', 'одиннадцатого').replace('twelve', 'двенадцатого').replace('thirteen', 'тринадцатого').replace('fourteen', 'четырнадцатого').replace('fifteen', 'пятнадцатого').replace('sixteen', 'шестнадцатого').replace('seventeen', 'семнадцатого').replace('eighteen', 'восемнадцатое').replace('nineteen', 'девятнадцатого').replace('twenty-one', 'двадцать первого').replace('twenty-two', 'двадцать второго').replace('twenty-three', 'двадцать третьего').replace('twenty-four', 'двадцать четвертого').replace('twenty-five', 'двадцать пятого').replace('twenty-six', 'двадцать шестого').replace('twenty-seven', 'двадцать седьмого').replace('twenty-eight', 'двадцать восьмого').replace('twenty-nine', 'двадцать девятого').replace('twenty', 'двадцатого').replace('thirty-one', 'тридцать первого').replace('thirty', 'тридцатого').replace('two', 'второго').replace('three', 'третьего').replace('four', 'четвертого').replace('five', 'пятого').replace('six', 'шестого').replace('seven', 'седьмого').replace('eight', 'восьмого').replace('nine', 'девятого').replace('one', 'первого').replace('ten', 'десятого')

results = pd.read_csv("../data/baseline_ext_ru.csv")
newColumn = results['after'].apply(dateFixer)
results['after'] = newColumn
results.to_csv(os.path.join(INPUT_PATH, "baseline_fixed_dates.csv"), index = False)

In [272]:
class SaarusTextNormalizer:

    def __init__(self):
        self.tokenizer = None
        self.model = None
        self.device = torch.device('cpu')
        self.re_tokens = re.compile(r"(?:[.,!?]|[а-яА-Я]\S*|\d\S*(?:\.\d+)?|[^а-яА-Я\d\s]+)\s*")
        
        self.roman_numerals = {
            'I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X',
            'XI', 'XII', 'XIII', 'XIV', 'XV', 'XVI', 'XVII', 'XVIII', 'XIX', 'XX',
            'XXI', 'XXII', 'XXIII', 'XXIV', 'XXV', 'XXVI', 'XXVII', 'XXVIII', 'XXIX', 'XXX',
            'XL', 'L', 'LX', 'LXX', 'LXXX', 'XC', 'C', 'CC', 'CCC', 'CD', 'D', 'DC', 'DCC', 'DCCC', 'CM', 'M'
        }
        

    def load_model(self):
        try:
            print("Загружаем модель saarus72/russian_text_normalizer...")
            
            self.tokenizer = GPT2Tokenizer.from_pretrained(
                "saarus72/russian_text_normalizer", 
                eos_token='</s>'
            )
            self.model = T5ForConditionalGeneration.from_pretrained(
                "saarus72/russian_text_normalizer"
            )
            self.model.to(self.device)
            self.model.eval()
            
            print(f"Модель загружена на {self.device}")
            return True
            
        except Exception as e:
            print(f"Ошибка при загрузке модели: {e}")
            return False

    def is_roman_numeral(self, text):
        text_upper = text.upper().strip()
        return text_upper in self.roman_numerals
    

    def tokenize(self, text):
        return re.findall(self.re_tokens, text)
    

    def strip_numbers(self, s):
        result = []
        for part in s.split():
            if part.isdigit():
                while len(part) > 3:
                    result.append(part[:- 3 * ((len(part) - 1) // 3)])
                    part = part[- 3 * ((len(part) - 1) // 3):]
                if part:
                    result.append(part)
            else:
                result.append(part)
        return " ".join(result)
    

    def construct_prompt(self, text):
        result = "<SC1>"
        etid = 0
        token_to_add = ""
        
        for token in self.tokenize(text) + [""]:
            if not re.search("[a-zA-Z\d]", token):
                if token_to_add:
                    end_match = re.search(r"(.+?)(\W*)$", token_to_add, re.M)
                    if end_match:
                        groups = end_match.groups()
                        result += f"[{self.strip_numbers(groups[0])}]<extra_id_{etid}>{groups[1]}"
                        etid += 1
                        token_to_add = ""
                result += token
            else:
                token_to_add += token
        
        return result

    def predict(self, text):
        try:
            input_ids = torch.tensor([self.tokenizer.encode(text)]).to(self.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids, 
                    max_new_tokens=50, 
                    eos_token_id=self.tokenizer.eos_token_id, 
                    early_stopping=True,
                    do_sample=False
                )
            
            return self.tokenizer.decode(outputs[0][1:])
            
        except Exception as e:
            print(f"Ошибка при предсказании для '{text}': {e}")
            return ""

    def construct_answer(self, prompt, prediction):
        re_prompt = re.compile(r"\[([^\]]+)\]<extra_id_(\d+)>")
        re_pred = re.compile(r"\<extra_id_(\d+)\>(.+?)(?=\<extra_id_\d+\>|</s>)")
        
        pred_data = {}
        for match in re.finditer(re_pred, prediction.replace("\n", " ")):
            pred_data[match[1]] = match[2].strip()
        
        while match := re.search(re_prompt, prompt):
            replace = pred_data.get(match[2], match[1])
            prompt = prompt[:match.span()[0]] + replace + prompt[match.span()[1]:]
        
        return prompt.replace("<SC1>", "")
    

    def normalize_text(self, text):
        if self.model is None or self.tokenizer is None:
            raise ValueError("Модель не загружена. Вызовите сначала load_model().")
        
        try:
            # Создаем промпт
            prompt = self.construct_prompt(text)
            
            # Получаем предсказание
            prediction = self.predict(prompt)
            
            # Строим финальный ответ
            answer = self.construct_answer(prompt, prediction)
            
            return answer.strip()
            
        except Exception as e:
            print(f"Ошибка при нормализации '{text}': {e}")
            return text

In [31]:
class My_TextNormalization_Model:
    def __init__(self):
        self.saarus_normalizer = SaarusTextNormalizer()
        self.is_loaded = False
    
    def load_model(self):
        self.is_loaded = self.saarus_normalizer.load_model()
        return self.is_loaded
    
    def normalize_text(self, text: str) -> str:

        if not self.is_loaded:
            print("Модель не загружена. Загружаем...")
            if not self.load_model():
                print("Не удалось загрузить модель. Возвращаем исходный текст.")
                return text
        
        return self.saarus_normalizer.normalize_text(text)
    
    def load_and_align_data(self, original_data_path, rule_based_output_path):
        try:
            original_data = pd.read_csv(original_data_path)
            print(f"Исходные данные: {len(original_data)} строк")
            
            original_data['composite_id'] = original_data['sentence_id'].astype(str) + '_' + original_data['token_id'].astype(str)
            
            rule_output = pd.read_csv(rule_based_output_path)
            print(f"Rule-based результаты: {len(rule_output)} строк")
            
            original_ids = set(original_data['composite_id'])
            rule_ids = set(rule_output['id'])
            
            common_ids = original_ids.intersection(rule_ids)
            print(f"Общих ID: {len(common_ids)}")
            
            if len(common_ids) == 0:
                print("Нет общих ID!")
                return None, None
            
            original_filtered = original_data[original_data['composite_id'].isin(common_ids)].copy()
            rule_filtered = rule_output[rule_output['id'].isin(common_ids)].copy()
            
            original_filtered = original_filtered.sort_values('composite_id').reset_index(drop=True)
            rule_filtered = rule_filtered.sort_values('id').reset_index(drop=True)
            
            print(f"После фильтрации: {len(original_filtered)} строк")
            
            return original_filtered, rule_filtered
            
        except Exception as e:
            print(f"Ошибка при загрузке и выравнивании данных: {e}")
            return None, None
    
    def contains_markers(self, text):
        return "_" in str(text)
    
    def needs_neural_processing(self, text):
        text_str = str(text).strip()
        if self.contains_markers(text_str):
            return False
        
        return bool(re.search(r'\d', text_str and r'(?![а-яА-Я])(?![a-zA-Z])') or re.search(r'thousand|hundred|fifty|million|billion', text_str))
    
    def postprocess_neural_result(self, original_text, neural_result):
        original_clean = str(original_text).strip()
        neural_clean = str(neural_result).strip()
        
        if re.match(r'^[а-яА-Я\s]+$', neural_clean) and not(re.match(r'\d', original_clean)) and not(re.match(r'[а-яА-Я]', original_clean)):
            if self.saarus_normalizer.is_roman_numeral(original_clean):
                return neural_clean
                
            letters = []
            for char in neural_clean.replace(' ', ''):
                if char.isalpha():
                    letters.append(f"{char.lower()}_trans")
                else:
                    letters.append(char)
            return ' '.join(letters)
        return neural_clean
    
    def improve_rule_based_results(self, original_data_path, rule_based_output_path, output_path, sample_size=None):
        print("Финальное улучшение rule_baseline")
        
        if not self.is_loaded:
            if not self.load_model():
                print("Не удалось загрузить модель")
                return
        
        original_data, rule_output = self.load_and_align_data(original_data_path, rule_based_output_path)
        
        if original_data is None or rule_output is None:
            print("Не удалось загрузить или выровнять данные")
            return
        
        if sample_size is not None and sample_size < len(rule_output):
            indices = rule_output.sample(n=sample_size, random_state=42).index
            rule_output = rule_output.loc[indices].reset_index(drop=True)
            original_data = original_data.loc[indices].reset_index(drop=True)
            print(f"Используем выборку из {sample_size} строк")
        
        print(f"Будет проанализировано {len(rule_output)} строк")
        
        texts_to_normalize = []
        indices_to_normalize = []
        
        with_marker_count = 0
        needs_neural_count = 0
        roman_numerals_count = 0
        
        for i in range(len(rule_output)):
            rule_result = rule_output.iloc[i]['after']
            
            if self.contains_markers(rule_result):
                with_marker_count += 1
            
            if self.needs_neural_processing(rule_result):
                if self.saarus_normalizer.is_roman_numeral(str(rule_result).strip()):
                    roman_numerals_count += 1
                    continue  # Пропускаем римские цифры
                
                needs_neural_count += 1
                texts_to_normalize.append(original_data.iloc[i]['before'])
                indices_to_normalize.append(i)
        
        print(f"Статистика анализа:")
        print(f"  - Строк с маркерами: {with_marker_count}")
        print(f"  - Римских цифр: {roman_numerals_count}")
        print(f"  - Строк для нейронной обработки: {needs_neural_count}")
        
        if not texts_to_normalize:
            print("Нет строк для нейронной обработки. Копируем исходные результаты.")
            rule_output.to_csv(output_path, index=False)
            return
        
        normalization_cache = {}
        
        print("Нормализация с помощью Saarus модели...")
        for text in tqdm(texts_to_normalize, desc="Нормализация"):
            try:
                normalized = self.normalize_text(text)
                
                final_result = self.postprocess_neural_result(text, normalized)
                
                normalization_cache[text] = final_result
                
            except Exception as e:
                print(f"Ошибка при нормализации '{text}': {e}")
                normalization_cache[text] = str(text)
        
        improved_results = rule_output.copy()
        changes_count = 0
        
        for i, text in zip(indices_to_normalize, texts_to_normalize):
            if text in normalization_cache:
                improved_text = normalization_cache[text]
                if improved_text != str(text):
                    improved_results.at[i, 'after'] = improved_text
                    changes_count += 1
        
        improved_results.to_csv(output_path, index=False)
        
        print(f"\nРезультаты сохранены в {output_path}")
        print(f"Всего улучшений: {changes_count}")
        
        print("\nПримеры улучшений:")
        shown = 0
        for original_text, improved_text in normalization_cache.items():
            if str(original_text) != str(improved_text) and shown < 15:
                print(f"  '{original_text}' -> '{improved_text}'")
                shown += 1

def run_final_pipeline(original_data_path, rule_based_output_path, output_path, sample_size=None):

    print("Запуск финализации результата")
    
    if not os.path.exists(original_data_path):
        print(f"Файл {original_data_path} не найден")
        return False
    
    if not os.path.exists(rule_based_output_path):
        print(f"Файл {rule_based_output_path} не найден")
        return False
    
    model = My_TextNormalization_Model()
    
    if not model.load_model():
        print("Не удалось загрузить модель")
        return False
    
    model.improve_rule_based_results(
        original_data_path=original_data_path,
        rule_based_output_path=rule_based_output_path,
        output_path=output_path,
        sample_size=sample_size
    )
    
    print(f"\n Готово. Результаты сохранены в {output_path}")
    return True

if __name__ == "__main__":
    run_final_pipeline(
        original_data_path="../data/ru_test_2.csv",
        rule_based_output_path="../data/baseline_fixed_dates.csv",
        output_path="../data/final_neural_submission.csv",
        #sample_size=1000
    )

Запуск финализации результата
Загружаем модель saarus72/russian_text_normalizer...
Модель загружена на cpu
Финальное улучшение rule_baseline
Исходные данные: 989880 строк
Rule-based результаты: 989880 строк
Общих ID: 989880
После фильтрации: 989880 строк
Будет проанализировано 989880 строк
Статистика анализа:
  - Строк с маркерами: 62041
  - Римских цифр: 0
  - Строк для нейронной обработки: 989
Нормализация с помощью Saarus модели...


Нормализация:   0%|                                                                    | 0/989 [00:00<?, ?it/s]C:\Users\user\AppData\Roaming\pypoetry\venv\Lib\site-packages\transformers\generation\configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Нормализация: 100%|██████████████████████████████████████████████████████████| 989/989 [18:39<00:00,  1.13s/it]



Результаты сохранены в ../data/final_neural_submission.csv
Всего улучшений: 989

Примеры улучшений:
  '1393887' -> 'миллион триста девяносто три тысячи восемьсот восемьдесят семь'
  '65340' -> 'шестьдесят пять тысяч триста сорок'
  '2232 с.' -> 'две тысячи двести тридцать два с.'
  '31 мая 1971 г.' -> 'тридцать одно мая тысяча девятьсот семьдесят первого г.'
  '9 октября 1983 год' -> 'девятое октября тысяча девятьсот восемьдесят третий год'
  '3700129890' -> 'три миллиона семьсот тысяч сто двадцать девять тысяч восемьсот девяносто'
  '25750460' -> 'двадцать пять миллионов семьсот пятьдесят тысяч четыреста шестьдесят'
  '43830' -> 'сорок три тысячи восемьсот тридцать'
  '1285402227' -> 'миллиард двести восемьдесят пять миллионов четыреста две тысячи двести двадцать семь'
  '363 Мбит/с' -> 'триста шестьдесят три Мбит/с'
  '22 461' -> 'двадцать две тысячи четыреста шестьдесят один'
  '12 декабря 1802 года' -> 'Двенадцатого декабря тысяча восемьсот второго года'
  '3315,4 км.' -> 'три тыс